In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
# Define the path to your dataset
dataset_path = '/content/drive/My Drive/Data Sets/Training-Data/train'


In [ ]:
num_classes = 13

In [ ]:
datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training')

Found 364 images belonging to 13 classes.


In [ ]:
val_generator = datagen.flow_from_directory(dataset_path, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation')

Found 85 images belonging to 13 classes.


In [ ]:
# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=10)


Epoch 1/10
12/12 [==============================] - 224s 18s/step - loss: 3.3485 - accuracy: 0.1923 - val_loss: 2.2943 - val_accuracy: 0.1529
Epoch 2/10
12/12 [==============================] - 58s 5s/step - loss: 2.3230 - accuracy: 0.1951 - val_loss: 2.2475 - val_accuracy: 0.2353
Epoch 3/10
12/12 [==============================] - 57s 5s/step - loss: 2.2684 - accuracy: 0.2253 - val_loss: 2.2354 - val_accuracy: 0.2353
Epoch 4/10
12/12 [==============================] - 60s 5s/step - loss: 2.2467 - accuracy: 0.2280 - val_loss: 2.2661 - val_accuracy: 0.2353
Epoch 5/10
12/12 [==============================] - 55s 5s/step - loss: 2.2126 - accuracy: 0.2308 - val_loss: 2.1925 - val_accuracy: 0.3647
Epoch 6/10
12/12 [==============================] - 54s 4s/step - loss: 2.1417 - accuracy: 0.2912 - val_loss: 2.0932 - val_accuracy: 0.3294
Epoch 7/10
12/12 [==============================] - 60s 5s/step - loss: 2.0437 - accuracy: 0.3159 - val_loss: 2.0257 - val_accuracy: 0.4235
Epoch 8/10
12/12 [

In [ ]:
# Save the trained model
model.save('trained_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Function to preprocess the image for the trained model
def preprocess_image(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values
    return img_array


In [ ]:
def predict_lettuce_type(img_path, confidence_threshold=0.1):
    img_array = preprocess_image(img_path)
    predictions = model.predict(img_array)
    class_index = np.argmax(predictions)
    confidence = predictions[0, class_index]

    if confidence >= confidence_threshold:
        if class_index == 0:
            return "Alternaria Leaf Spot"
        elif class_index == 1:
            return "Angular Leaf Spot"
        elif class_index == 2:
            return "Anthracnose"
        elif class_index == 3:
            return "Bacterial Leaf Spot"
        elif class_index == 4:
            return "Black Rot Leaf Spot"
        elif class_index == 5:
            return "Cercospora Leaf Spot"
        elif class_index == 6:
            return "Downy Mildew"
        elif class_index == 7:
            return "Gummy Stem Blight"
        elif class_index == 8:
            return "Healthy"
        elif class_index == 9:
            return "Mosaic Virus"
        elif class_index == 10:
            return "Powdery Mildew"
        elif class_index == 11:
            return "Rust"
        elif class_index == 2:
            return "Septoria Leaf Spot"
    else:
        return "Unknown"

In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 38.4 MB/s eta 0:00:00


In [ ]:
from dash import Dash, dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate
import base64

In [ ]:
app = Dash(__name__)

In [ ]:
app.layout = html.Div([
    html.H1("Lettuce Type Prediction"),
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        multiple=False
    ),
    html.Div(id='output-image-upload'),
])

In [ ]:
@app.callback(
    Output('output-image-upload', 'children'),
    [Input('upload-image', 'contents')],
    [State('upload-image', 'filename')],
)
def update_output(contents, filename):
    if contents is None:
        raise PreventUpdate

    try:
        # Save the uploaded image
        img_data = contents.split(',')[1]
        img_data_decoded = base64.b64decode(img_data)
        img_path = 'uploaded_image.jpg'
        with open(img_path, 'wb') as f:
            f.write(img_data_decoded)

        # Get the predicted lettuce type
        lettuce_type = predict_lettuce_type(img_path)

        # Display the result
        return [
            html.H5(f'Uploaded Image: {filename}'),
            html.Img(src=contents),
            html.H6(f'Predicted Lettuce Type: {lettuce_type}')
        ]

    except Exception as e:
        return f'Error: {str(e)}'

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>